# Compare Training Hyperparameters

In [ ]:
! git clone https://github.com/MaxHReinhardt/ArchitecturalStyleClassification

In [2]:
import sys
import os

py_file_location = "/content/ArchitecturalStyleClassification/src"
sys.path.append(os.path.abspath(py_file_location))

In [3]:
import torch
import numpy as np

from preprocessing import TrainSetDynamicNormalization, EvaluationSetStaticNormalization
from model import MobileNetV1
from train_model import train_with_early_stopping
from evaluate_model import evaluate


def compare_training_hyperparameter_configurations(learning_rate_range, batch_size_range, num_configurations, train_csv,
                                                   validation_csv):
    """
    Performs random search for given ranges of training hyperparameters.
    """

    weight_decay = 0
    max_num_epochs = 100

    resolution = 384
    width_multiplier = 1
    cbam_last_layer = False
    cbam_all_layers = True

    # Check if CUDA (GPU) is available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device: {device}")

    for _ in range(num_configurations):
        # Randomly select learning rate and batch size from the provided ranges
        learning_rate = np.power(10,
                                 np.random.uniform(np.log10(learning_rate_range[0]), np.log10(learning_rate_range[1])))
        batch_size = int(np.power(2, np.random.uniform(np.log2(batch_size_range[0]), np.log2(batch_size_range[1]))))
        print(f"Learning rate: {learning_rate}, batch size: {batch_size}.")

        train_set_object = TrainSetDynamicNormalization(resolution=resolution,
                                                        train_csv=train_csv)
        train_set = train_set_object.get_data()
        normalization_mean, normalization_std = train_set_object.get_normalization_parameters()
        validation_set = EvaluationSetStaticNormalization(resolution=resolution,
                                                          evaluation_csv=validation_csv,
                                                          normalization_mean=normalization_mean,
                                                          normalization_std=normalization_std).get_data()

        model = MobileNetV1(ch_in=3, n_classes=25, width_multiplier=width_multiplier,
                            cbam_all_layers=cbam_all_layers, cbam_last_layer=cbam_last_layer)
        model.to(device)

        trained_model, train_loss_development, val_loss_development = train_with_early_stopping(model,
                                                                                                train_set,
                                                                                                validation_set,
                                                                                                batch_size,
                                                                                                learning_rate,
                                                                                                max_num_epochs,
                                                                                                device,
                                                                                                weight_decay)

        model_name = f"{width_multiplier}-MobileNetV1-{resolution}_cbam_all_layers_lr-{learning_rate}_bs-{batch_size}"
        model_path = os.path.join("stored_models/", model_name + ".pth")
        torch.save(trained_model.state_dict(), model_path)

        accuracy, _, avg_loss, avg_prediction_time = evaluate(trained_model,
                                                              validation_set,
                                                              batch_size,
                                                              device)

        print(f"{model_name} evaluation -- Accuracy: {accuracy}, Average loss: {avg_loss}, "
              f"Average prediction time (seconds): {avg_prediction_time}")

In [4]:
!mkdir stored_models

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
!unzip /content/gdrive/MyDrive/data.zip;

In [7]:
drive.flush_and_unmount()

## Run Experiment

In [8]:
learning_rate_range = [0.002, 0.005]
batch_size_range = [32, 128]
num_configurations = 10
train_csv = "data/dataset/train_annotation.csv"
validation_csv = "data/dataset/validation_annotation.csv"

In [ ]:
compare_training_hyperparameter_configurations(learning_rate_range, batch_size_range, num_configurations, train_csv, validation_csv)